In [1]:
import torch

In [2]:
t1=torch.tensor(4.)
t1

tensor(4.)

In [3]:
t1.dtype

torch.float32

In [4]:
t2=torch.tensor([1.,2,3,4])
t2

tensor([1., 2., 3., 4.])

In [5]:
t2.dtype

torch.float32

In [6]:
t3=torch.tensor([[5.,6],
                 [7,8],
                 [9,10]])
t3

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])

In [7]:
t6=torch.full((3,2),42)

In [8]:
t7 = torch.cat ((t3,t6))
t7

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.],
        [42., 42.],
        [42., 42.],
        [42., 42.]])

In [9]:
t8= torch.sin(t7)
t8

tensor([[-0.9589, -0.2794],
        [ 0.6570,  0.9894],
        [ 0.4121, -0.5440],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165]])

In [10]:
t9=t8.reshape(3,2,2)
t9

tensor([[[-0.9589, -0.2794],
         [ 0.6570,  0.9894]],

        [[ 0.4121, -0.5440],
         [-0.9165, -0.9165]],

        [[-0.9165, -0.9165],
         [-0.9165, -0.9165]]])

## Linear Regression

In [19]:
import numpy as np
import torch

### Training data

In [20]:
# Inputs (temp , rainfall, humudity)
inputs= np.array([[73,67,43],
                 [91,88,64],
                 [87,134,58],
                 [102,43,37],
                 [69,96,70]] , dtype='float32')
                 

In [21]:
# Targets (apples, oranges)
targets = np.array([[56,70],
                    [81,10],
                    [119,133],
                    [22,37],
                    [103,119]], dtype='float32')

### Convert these arrays into torch tensors

In [22]:
inputs=torch.from_numpy(inputs)
targets= torch.from_numpy(targets)

#### Linear resression model from scratch

In [23]:
w= torch.randn(2,3 , requires_grad= True)
b= torch.randn(2, requires_grad= True)
print(w)
print(b)

tensor([[-0.7681, -0.2292, -1.3162],
        [-1.1405, -0.8850,  0.2348]], requires_grad=True)
tensor([-1.0110,  0.7186], requires_grad=True)


In [25]:
inputs @ w.t() + b

tensor([[-129.0394, -131.7388],
        [-175.3196, -165.9228],
        [-174.8940, -203.4771],
        [-137.9172, -144.9842],
        [-168.1512, -146.5020]], grad_fn=<AddBackward0>)

In [26]:
def model(x):
    return x @ w.t()+b

In [27]:
preds= model(inputs)
print(preds)

tensor([[-129.0394, -131.7388],
        [-175.3196, -165.9228],
        [-174.8940, -203.4771],
        [-137.9172, -144.9842],
        [-168.1512, -146.5020]], grad_fn=<AddBackward0>)


In [28]:
print(targets)

tensor([[ 56.,  70.],
        [ 81.,  10.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


### Loss Function

In [32]:
#MSE
def mse(t1,t2):
    diff= t1-t2
    return torch.sum(diff * diff)/diff.numel()

In [34]:
loss= mse(preds,targets)
print(loss)

tensor(57388.3242, grad_fn=<DivBackward0>)


### Compute Gradients

In [35]:
loss.backward()

In [37]:
print(w)
print(w.grad)

tensor([[-0.7681, -0.2292, -1.3162],
        [-1.1405, -0.8850,  0.2348]], requires_grad=True)
tensor([[-19484.5469, -21448.5059, -13260.9062],
        [-19378.2871, -21479.8281, -12953.6104]])


In [38]:
print(b)
print(b.grad)

tensor([-1.0110,  0.7186], requires_grad=True)
tensor([-233.2643, -232.3250])


In [40]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [41]:
print(w)
print(w.grad)

tensor([[-0.5733, -0.0147, -1.1836],
        [-0.9468, -0.6702,  0.3643]], requires_grad=True)
tensor([[-19484.5469, -21448.5059, -13260.9062],
        [-19378.2871, -21479.8281, -12953.6104]])


In [42]:
w , b

(tensor([[-0.5733, -0.0147, -1.1836],
         [-0.9468, -0.6702,  0.3643]], requires_grad=True),
 tensor([-1.0086,  0.7209], requires_grad=True))

In [44]:
preds=model(inputs)
loss=mse(preds,targets)
print(loss)

tensor(38993.7422, grad_fn=<DivBackward0>)


In [45]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


### Train the model using gradient descent

Implement the above step by step

In [52]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ -66.5898,  -69.6335],
        [ -93.2083,  -84.2996],
        [ -77.6693, -106.6585],
        [ -76.0254,  -83.4764],
        [ -89.2565,  -68.0823]], grad_fn=<AddBackward0>)


In [53]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(26596.7559, grad_fn=<DivBackward0>)


In [48]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-15942.6016, -17640.2891, -10911.3184],
        [-15855.7373, -17691.7031, -10616.8633]])
tensor([-191.2432, -190.5339])


Update the weights and biases using gradients computed above

In [49]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [51]:
print(w)
print(b)

tensor([[-0.4139,  0.1617, -1.0745],
        [-0.7882, -0.4932,  0.4705]], requires_grad=True)
tensor([-1.0067,  0.7228], requires_grad=True)


### Train for multiple epochs

In [130]:
# train for 100 epochs
for i  in range(10000):
    preds = model(inputs)
    loss = mse(preds,targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()


In [131]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(541.9397, grad_fn=<DivBackward0>)


In [132]:
preds

tensor([[ 56.9979,  54.7824],
        [ 82.3117,  69.9921],
        [118.7203, 132.4053],
        [ 21.1024,  18.8245],
        [101.8937,  83.5988]], grad_fn=<AddBackward0>)

In [133]:
targets

tensor([[ 56.,  70.],
        [ 81.,  10.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

### Linear Regression using Pytorch Inbuilts

In [134]:
import torch.nn as nn

In [135]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [136]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [137]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

### Dataset and DataLoaders

In [138]:
from torch.utils.data import TensorDataset

In [139]:
# Define dataset
train_ds =  TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [140]:
from torch.utils.data import DataLoader

In [141]:
# Define data loader
batch_size =5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [143]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [142]:
for xb,yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 91.,  87.,  65.],
        [ 68.,  97.,  70.],
        [103.,  43.,  36.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.]])
tensor([[ 80., 102.],
        [102., 120.],
        [ 20.,  38.],
        [ 21.,  38.],
        [104., 118.]])


### nn.Linear

In [144]:
# Define model
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.0538, -0.2831,  0.2875],
        [ 0.0245,  0.5101,  0.3232]], requires_grad=True)
Parameter containing:
tensor([0.1814, 0.2524], requires_grad=True)


In [145]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.0538, -0.2831,  0.2875],
         [ 0.0245,  0.5101,  0.3232]], requires_grad=True),
 Parameter containing:
 tensor([0.1814, 0.2524], requires_grad=True)]

In [146]:
# Generate Predictions
preds = model(inputs)
preds

tensor([[-10.3524,  50.1141],
        [-11.2289,  68.0535],
        [-25.7589,  89.4798],
        [ -6.8450,  36.6448],
        [-10.5843,  73.5333],
        [-10.1232,  49.6285],
        [-10.6584,  67.8666],
        [-25.5253,  89.8275],
        [ -7.0742,  37.1304],
        [-10.2430,  73.8319],
        [ -9.7819,  49.9271],
        [-10.9997,  67.5680],
        [-26.3294,  89.6667],
        [ -7.1863,  36.3462],
        [-10.8135,  74.0188]], grad_fn=<AddmmBackward0>)

In [148]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

## Loss Function

In [149]:
# Import nn.functional
import torch.nn.functional as F

In [151]:
loss_fn = F.mse_loss

In [152]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(5279.2505, grad_fn=<MseLossBackward0>)


## Optimizer

In [153]:
# Define Optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

## Train the model

In [156]:
# Utility function to train the model

def fit(num_epochs,model, loss_fn, opt, train_dl):

    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        for xb,yb in train_dl:
        # 1. generate predictions
            pred = model(xb)
        # 2. calculate loss
            loss = loss_fn(pred,yb)
        # 3. Compute Gradients
            loss.backward()
        # 4. upate parameters using gradients
            opt.step()
        # 5. Reset the gradients to zero
            opt.zero_grad()
        # print the progress
        if (epoch+1)%10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs} | Loss: {loss.item()}")

Train the model for 100 epochs

In [157]:
fit(100, model, loss_fn,opt,train_dl)

Epoch 10/100 | Loss: 51.253257751464844
Epoch 20/100 | Loss: 61.489585876464844
Epoch 30/100 | Loss: 35.4142951965332
Epoch 40/100 | Loss: 24.996383666992188
Epoch 50/100 | Loss: 12.8175630569458
Epoch 60/100 | Loss: 18.968666076660156
Epoch 70/100 | Loss: 13.720434188842773
Epoch 80/100 | Loss: 13.267007827758789
Epoch 90/100 | Loss: 11.526849746704102
Epoch 100/100 | Loss: 9.603827476501465


In [158]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.4053,  70.8783],
        [ 83.2077,  98.8224],
        [114.7197, 136.3083],
        [ 23.4264,  39.8045],
        [102.4239, 114.3409],
        [ 56.2686,  69.7864],
        [ 83.2504,  98.5773],
        [115.1379, 136.7295],
        [ 24.5631,  40.8964],
        [103.6034, 115.1877],
        [ 57.4480,  70.6331],
        [ 82.0709,  97.7305],
        [114.6770, 136.5535],
        [ 22.2469,  38.9577],
        [103.5607, 115.4328]], grad_fn=<AddmmBackward0>)

In [159]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [160]:
model(torch.tensor([[75,63,44.]]))

tensor([[54.4183, 67.5702]], grad_fn=<AddmmBackward0>)